In [16]:
#imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
from collections import deque
from selenium.webdriver.common.keys import Keys
import time 

In [17]:
#Chromedriver
browser = webdriver.Firefox()
browser.get('https://web.telegram.org/#/im')

In [18]:
#Lets the user select groups to track
def selectGroups(groupChats):
    index = 1
    for chat in groupChats:
        title= chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        print(str(index) + '. ' + title)
        index+=1
    
    #once chats are printed, ask user for csv of indexes of groupchats they want to track
    listOfIndexOfChats = []
    while True:
        try:
            groupNumber = int(input("Please input the number of"+
                                    " the groupchat you want to track (any non-number to proceed): "))
            listOfIndexOfChats.append(groupNumber)
        except ValueError:
            break
            
    #now that we have the group indeces, use global variable to change indeces to chatNames
    userChats = []
    for index in listOfIndexOfChats:
        userChats.append(groupChats[index-1])
    print('You have selected the following groups: ')
    #return userChats
    return userChats
    
#________________________________________________________________________#
def selectHomeGroup(groupChats):
    index = int(input("Please input the number of the Home Group: "))
    homeGroup = groupChats[index-1].find_elements_by_css_selector("div.im_dialog_peer")[0].text
    print('Selected '+ homeGroup+' as homegroup.')
    return homeGroup

#________________________________________________________________________#
#Creates dictionairy of last messages in each group
def initializeDict(groupChats):
    lastMessageDict = {}
    for chat in groupChats:
        chat.click()
        selectedGroupTitle = chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        while True:
            try:
                messages = browser.find_elements_by_class_name("im_message_text")
                textMessages = []
                #change array of WebElements to array of strings. easier to work with, avoid errors
                for webElement in messages:
                    textMessages.append(webElement.text)
                #for some reason there were empty strings being grabbed by selenium. removing them here    
                textMessages = list(filter(None, textMessages))
                key_gc = selectedGroupTitle
                lastMessageDict[key_gc] = textMessages[-1]
                print('Loaded ' + selectedGroupTitle + ': '+ lastMessageDict[key_gc])
                break
            except:
                print(selectedGroupTitle + ' did not load yet, retrying.')
    print(lastMessageDict)
    print('Recorded most recent messages in every group. New messages will be pushed to homegroup.')
    return lastMessageDict
#________________________________________________________________________#
#groupQueueBuilder
def groupQueueBuilder(trackedChats):
    for groupChat in trackedChats:
        oneBadgeArray = groupChat.find_elements_by_css_selector("span.im_dialog_badge.badge")
#         TO DO : make it not take messages from HOME GROUP
        if(oneBadgeArray[0].text!=""):
            #get index and use this index in titlesArray
            oneTitleArray = groupChat.find_elements_by_css_selector("div.im_dialog_peer")
            if(oneTitleArray[0].text not in groupQueue):
                groupQueue.append(oneTitleArray[0].text)
    print(groupQueue)
    return groupQueue
#________________________________________________________________________#
#groupFinder - Takes in name of group finds it selects it, returns title of slected chat
def groupFinder(name, trackedChats):
    print('Finding '+ name)
    for chat in trackedChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == name):
            chat.click()
            print('Clicked on ' + name)
            return(titleArray[0].text)
    print('groupFinder Could not find group: '+ name)
    return 'noGroupError'
    
#________________________________________________________________________#
#Create push method to click on 'home chat' and paste and send it there.
# checks via dictionary which messages to push

def createMessageStack(selectedGroupTitle, lastMessageDict):
    linkedGroupChatContainer = browser.find_element_by_class_name('im_history_loaded')
#     print(linkedGroupChatContainer)
    groupChatContainerElement = linkedGroupChatContainer.find_elements_by_class_name("im_history_messages_peer")
#     print(groupChatContainerElement)
    for element in groupChatContainerElement:
        if(element.get_attribute("class") == 'im_history_messages_peer'): 
            messages = element.find_elements_by_class_name("im_message_text")
#             print(messages)
    textMessages = []
    print('Loading '+str(len(messages))+' new messages')
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
    #     if(len(webElement.text) > 1):
#         print(webElement.text)
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    textMessages.reverse()
    ##
    messageStack = []
    for message in textMessages:
        if(message != lastMessageDict[selectedGroupTitle]):
            messageStack.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            break
    print((str)(len(messageStack)) + ' messages Loaded')
    print(messageStack)
    return messageStack
    ##
    print(textMessages)

#DEPRECATED METHOD
# def createMessageStack(selectedGroupTitle, lastMessageDict):
# #     groupChatContainerElement = browser.find_element_by_class_name("im_history_messages_peer")
#     messageStack = []
#     messages = browser.find_elements_by_class_name("im_message_text")
#     textMessages = []
#     print('Loading '+str(len(messages))+' new messages')
#     #change array of WebElements to array of strings. easier to work with, avoid errors
#     for webElement in messages:
#         if(len(webElement.text) > 1):
#             print(webElement.text)
#         textMessages.append(webElement.text)
#     #for some reason there were empty strings being grabbed by selenium. removing them here    
#     textMessages = list(filter(None, textMessages))
#     #reversing list to make FIFO
#     textMessages.reverse()
#     for message in textMessages:
#         if(message != lastMessageDict[selectedGroupTitle]):
#             messageStack.append(message)
#         else:
# #             SHOULD RUN WHEN MESSAGES ARE PUSHED
#             break
#     print((str)(len(messageStack)) + ' messages Loaded')
#     print(messageStack)
#     return messageStack
 #________________________________________________________________________#
# Broadcast Method
def broadCast(messageStack, selectedGroupTitle, groupChats, homeGroup, dictionairy):
    #select home chat
    print('Sending...')
    print(messageStack)
    messageStack.reverse()
    for chat in groupChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == homeGroup):
            chat.click()
            break
    
    #once homechat is selected, send messages one by one.
    textBox = browser.find_elements_by_class_name('composer_rich_textarea')[0]
    for message in messageStack:
        try:
            textBox.send_keys(selectedGroupTitle + ": " + message)
        except:
            print('*****THIS IS THE MESSAGE***** ' + message)
            #TODO HERE WE NEED TO STRIP THE MESSAGE OF THE EMOJIS AND RESEND
            pass
        textBox.send_keys(Keys.RETURN)
    if(len(messageStack) != 0):
        dictionairy[selectedGroupTitle] = messageStack[-1]
    #clean up after sending messages.
    messageStack.clear()
    print('Messages sent!')
    #print(messageStack)
    return 0
#________________________________________________________________________#
def fillGroupQueue(trackedChats):
    groupQueue = deque([])
    for webElement in trackedChats:
        titleArray = webElement.find_elements_by_css_selector("div.im_dialog_peer")
        groupQueue.append(titleArray[0].text)
    print('Time to check every group for missed messages')
    print(groupQueue)
    return groupQueue
        
#________________________________________________________________________#
#User prompt to run BotLoop
global READY
READY = False

while True:
    print('______________________________________________________________')
    print('READ DIRECTIONS CAREFULLY.')
    try:
        userPrompt = input("Please type 'ready' after signing in to continue. ")            
        
    except ValueError:
        print("You gotta type in 'ready' exactly like that, bud...")
        continue

    else:
        if(userPrompt == 'ready'):
            READY = True
            break
        else:
            continue

#________________________________________________________________________#
#BotLoop
#grabs all groupchats

# while True:
#     print('inside botLoop')
# try:
if (READY):
    #Create Runtime Variables inside here.
    groupChats = browser.find_elements_by_class_name("im_dialog_wrap") 
    groupQueue = deque([])
    trackedChats = selectGroups(groupChats)
    homeGroup = selectHomeGroup(groupChats)
    lastMessageDict = initializeDict(trackedChats)
    timeout = 3600
    timeout_start = time.time()
    cleanupTimeout = time.time()
    # month timeout:
    # while time.time() < 1517278786 + 259200:
    while time.time() < timeout_start + timeout:
        if(len(list(groupQueue)) == 0):
            time.sleep(1)
            if(time.time() > cleanupTimeout + 10):
                groupQueue = fillGroupQueue(trackedChats)
                cleanupTimeout = time.time()
            else:
                groupQueue = groupQueueBuilder(trackedChats)
                if(len(list(groupQueue)) != 0):
                    cleanupTimeout = time.time()
                
        else:
            selectedGroupTitle = groupFinder(groupQueue.popleft(), trackedChats)
            messageStack = createMessageStack(selectedGroupTitle, lastMessageDict)
            broadCast(messageStack, selectedGroupTitle, groupChats,homeGroup, lastMessageDict)
            cleanupTimeout = time.time()

    print('Finished BotLoop')
    
    
        
        
            #get first in queue and cache / print all messages from it.
            #For our purpose, batch work scheduler would be better. **prioritizes active groupchats**
    # except:
        # pass


        

______________________________________________________________
READ DIRECTIONS CAREFULLY.
Please type 'ready' after signing in to continue. ready
1. ICOcountdown.com - http://telegram.me/icocountdown
2. Refereum
3. The Coin Farm
4. WhaleClub (Bitcoin ONLY) Traders
5. Alt Whales
6. www.therealtycoin.io
7. FGCU CRYPTO CRIPS
8. Crypto Hours
9. PAID GROUP
10. Telegram
11. Sean Velez
12. Jakub Nowak
13. Danny A
14. RefereumBot
15. Whale Group
16. Telecast
17. Anton Invar
18. jerry lin
Please input the number of the groupchat you want to track (any non-number to proceed): 1
Please input the number of the groupchat you want to track (any non-number to proceed): 2
Please input the number of the groupchat you want to track (any non-number to proceed): 3
Please input the number of the groupchat you want to track (any non-number to proceed): 4
Please input the number of the groupchat you want to track (any non-number to proceed): 5
Please input the number of the groupchat you want to track (any n

Messages sent!
Finding Refereum
Clicked on Refereum
Loading 39 new messages
39 messages Loaded
['And gets dumped very hard', "you'll see once the tokens are distributed", 'Thank You admin :)', 'I hope it fails', 'again not true', "I'm leaving this shit coin", 'They used everyone hype to sell out to whales', '@LoveCrypto', 'Period', 'They have made the system more refined against fake botting or whatever unwanted practices are done.', 'But to all of us .  Everyone will think this as a scam.', 'Who can I contact so that I change my wallet in the table?', "yes that's more accurate, the team had to make hard choices and they know about the many displeased people in the community", "You can say what you want, but we don't believe you.  I personally have been waiting and earning points on the site since October.  This went to whales, and it will show when it hits exchange...  I guess I will go create a TON of fake telegram, DISCORD, and Twitch accounts so I can take my way to top points.", "

Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 20 new messages
1 messages Loaded
['bot']
Sending...
['bot']
Messages sent!
Finding Alt Whales
Clicked on Alt Whales
Loading 3 new messages
3 messages Loaded
['Wait imo', 'Its gonma drop lower', '?']
Sending...
['Wait imo', 'Its gonma drop lower', '?']
Messages sent!
deque(['Refereum', 'WhaleClub (Bitcoin ONLY) Traders'])
Finding Refereum
Clicked on Refereum
Loading 40 new messages
1 messages Loaded
["It's a great coin/project, we just got played by Refereum and Dylan.  Used to hype the coin, then sold to Whales.    It happens.   Just stop trying to insult our intelligence.  Moving on."]
Sending...
["It's a great coin/project, we just got played by Refereum and Dylan.  Used to hype the coin, then sold to Whales.    It happens.   Just stop trying to insult our intelligence.  Moving on."]
Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Tr

1 messages Loaded
['Simple analogy.  \nIf they are false advertising at this very first stage of the coin. It will be a 100% sure that they will make false promises again and will just pump and dump hopefully this project will fail.']
Sending...
['Simple analogy.  \nIf they are false advertising at this very first stage of the coin. It will be a 100% sure that they will make false promises again and will just pump and dump hopefully this project will fail.']
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
Time to check every group for missed messages
deque(['ICOcountdown.com - http://telegram.me/icocountdown', 'Refereum', 'The Coin Farm', 'WhaleClub (Bitcoin ONLY) Traders', 'Alt Whales'])
Finding ICOcountdown.com - http://telegram.me/icocountdown
Clicked on ICOcountdown.com - http://telegram.me/icocountdown
Loading 20 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Refereum
Clicked on Refereum
Loading 42 ne

Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
Time to check every group for missed messages
deque(['ICOcountdown.com - http://telegram.me/icocountdown', 'Refereum', 'The Coin Farm', 'WhaleClub (Bitcoin ONLY) Traders', 'Alt Whales'])
Finding ICOcountdown.com - http://telegram.me/icocountdown
Clicked on ICOcountdown.com - http://telegram.me/icocountdown
Loading 20 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Refereum
Clicked on Refereum
Loading 44 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding The Coin Farm
Clicked on The Coin Farm
Loading 31 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 13 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Alt Whales
Clicked on Alt Whales
Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque([

Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Refereum
Clicked on Refereum
Loading 45 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding The Coin Farm
Clicked on The Coin Farm
Loading 33 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 16 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Alt Whales
Clicked on Alt Whales
Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque(['The Coin Farm', 'WhaleClub (Bitcoin ONLY) Traders'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 33 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 17 new messages
1 messages Loaded
['Losing steam']
Sending...
['Losing steam']
Messages sent!
d

Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 40 new messages
1 messages Loaded
["I'll probably invest in XXX"]
Sending...
["I'll probably invest in XXX"]
Messages sent!
deque(['ICOcountdown.com - http://telegram.me/icocountdown'])
Finding ICOcountdown.com - http://telegram.me/icocountdown
Clicked on ICOcountdown.com - http://telegram.me/icocountdown
Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
Time to check every group for missed messages
deque(['ICOcountdown.com - http://telegram.me/icocountdown', 'Refereum', 'The Coin Farm', 'WhaleClub (Bitcoin ONLY) Traders', 'Alt Whales'])
Finding ICOcountdown.com - http://telegram.me/icocountdown
Clicked on ICOcountdown.com - http://telegram.me/icocountdown
Loading 23 new messages
0 messages Loaded
[]
Sending...
[]
Messages

0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Refereum
Clicked on Refereum
Loading 46 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding The Coin Farm
Clicked on The Coin Farm
Loading 42 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 21 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding Alt Whales
Clicked on Alt Whales
Loading 24 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque(['WhaleClub (Bitcoin ONLY) Traders'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 22 new messages
1 messages Loaded
['that summary screams "wait for us to shake out the public by pushing price down in 2018"']
Sending...
['that summary screams "wait for us to shake out the public by pushing price down in 2018"']
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
d

UnboundLocalError: local variable 'messages' referenced before assignment

In [8]:
def createMessageStack(selectedGroupTitle, lastMessageDict):
    linkedGroupChatContainer = browser.find_element_by_class_name('im_history_loaded')
#     print(linkedGroupChatContainer)
    groupChatContainerElement = linkedGroupChatContainer.find_elements_by_class_name("im_history_messages_peer")
#     print(groupChatContainerElement)
    for element in groupChatContainerElement:
        if(element.get_attribute("class") == 'im_history_messages_peer'): 
            messages = element.find_elements_by_class_name("im_message_text")
#             print(messages)
    textMessages = []
    print('Loading '+str(len(messages))+' new messages')
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
    #     if(len(webElement.text) > 1):
#         print(webElement.text)
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    textMessages.reverse()
    ##
    messageStack = []
    for message in textMessages:
        if(message != lastMessageDict[selectedGroupTitle]):
            messageStack.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            break
    print((str)(len(messageStack)) + ' messages Loaded')
    print(messageStack)
    return messageStack
    ##
    print(textMessages)

Loading 44 new messages
42 messages Loaded
["I think I've learn more from Tai Lopez", 'I thought this was a serious telegram :persevere:', 'Wait for 2000$', '7000$ is coming', 'If its not now then why there is soo many bitcoin atm that been distributed around the banks!!!!', 'Get an avatar please', 'lol', 'A lot of retards too', 'however, priob just scammers who have shorts on atm', 'Nevery seen so many scared ppl in  a single chat room before', 'I think so too but not now. So I short it', 'Thats something i cant predict but by the end of 2018 or begining of 2019 yes', 'you must be make some profit this very moment', 'Good point but do you think the time is now?', 'so I guess when there´s a crash, people think it will be back in a couple of days.... but those days might become a couple of years :expressionless:', 'Welcome Mikhail, please read the rules and introduction on https://whaleclub.io/whaleclub-main-rooms/', 'They need a gold standart. In the digitalized money. Just like real g